In [1]:
import pandas as pd
import numpy as np

In [3]:
barts_hotspots = pd.read_csv('data/barts_hotspots.csv')
barts_to_all = pd.read_csv('data/barts_to_all.csv')
hotspots_to_all = pd.read_csv('data/hotspots_to_all.csv')
hours_q1 = pd.read_csv('data/hours_q1.csv')
hours_q2 = pd.read_csv('data/hours_q2.csv')

In [5]:
barts_hotspots.to_csv('barts_hotspots_updated.csv')

In [23]:
barts_hotspots.head()

,Date,Origin Movement ID,Origin Display Name,Destination Movement ID,Destination Display Name,Daily Mean Travel Time (Seconds),Daily Range - Lower Bound Travel Time (Seconds),Daily Range - Upper Bound Travel Time (Seconds),AM Mean Travel Time (Seconds),AM Range - Lower Bound Travel Time (Seconds),...,Midday Mean Travel Time (Seconds),Midday Range - Lower Bound Travel Time (Seconds),Midday Range - Upper Bound Travel Time (Seconds),Evening Mean Travel Time (Seconds),Evening Range - Lower Bound Travel Time (Seconds),Evening Range - Upper Bound Travel Time (Seconds),Early Morning Mean Travel Time (Seconds),Early Morning Range - Lower Bound Travel Time (Seconds),Early Morning Range - Upper Bound Travel Time (Seconds),weekday
0,2019-06-09,3396,"The Palace Of Fine Arts, 3601 Lyon St, San Fra...",3603,"Embarcadero, San Francisco, CA",1588.0,1054.0,2392.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
1,2019-04-28,3603,"Embarcadero, San Francisco, CA",3394,"Fisherman's Wharf, San Francisco, CA",639.0,512.0,796.0,NaN,NaN,...,654.0,524.0,817.0,608.0,492.0,753.0,NaN,NaN,NaN,6
2,2019-05-18,3603,"Embarcadero, San Francisco, CA",3394,"Fisherman's Wharf, San Francisco, CA",730.0,520.0,1024.0,NaN,NaN,...,791.0,590.0,1061.0,569.0,406.0,797.0,NaN,NaN,NaN,5
3,2019-04-18,3792,"Oracle Park, 24 Willie Mays Plaza, San Francis...",3603,"Embarcadero, San Francisco, CA",512.0,371.0,705.0,648.0,461.0,...,528.0,398.0,702.0,442.0,321.0,608.0,363.0,297.0,443.0,3
4,2019-04-08,3792,"Oracle Park, 24 Willie Mays Plaza, San Francis...",3603,"Embarcadero, San Francisco, CA",505.0,364.0,701.0,621.0,489.0,...,429.0,339.0,542.0,502.0,354.0,711.0,NaN,NaN,NaN,0


In [28]:
barts_hotspots.sort_values('Date')

,Date,Origin Movement ID,Origin Display Name,Destination Movement ID,Destination Display Name,Daily Mean Travel Time (Seconds),Daily Range - Lower Bound Travel Time (Seconds),Daily Range - Upper Bound Travel Time (Seconds),AM Mean Travel Time (Seconds),AM Range - Lower Bound Travel Time (Seconds),...,Midday Mean Travel Time (Seconds),Midday Range - Lower Bound Travel Time (Seconds),Midday Range - Upper Bound Travel Time (Seconds),Evening Mean Travel Time (Seconds),Evening Range - Lower Bound Travel Time (Seconds),Evening Range - Upper Bound Travel Time (Seconds),Early Morning Mean Travel Time (Seconds),Early Morning Range - Lower Bound Travel Time (Seconds),Early Morning Range - Upper Bound Travel Time (Seconds),weekday
2106,2019-01-01,3792,"Oracle Park, 24 Willie Mays Plaza, San Francis...",3603,"Embarcadero, San Francisco, CA",684.0,381.0,1228.0,NaN,NaN,...,432.0,338.0,551.0,NaN,NaN,NaN,851.0,478.0,1515.0,1
2524,2019-01-01,3792,"Oracle Park, 24 Willie Mays Plaza, San Francis...",3692,2nd Street and Stevenson Street (Montgomery BA...,483.0,319.0,729.0,NaN,NaN,...,464.0,343.0,627.0,427.0,322.0,565.0,493.0,312.0,777.0,1
2091,2019-01-01,3603,"Embarcadero, San Francisco, CA",3396,"The Palace Of Fine Arts, 3601 Lyon St, San Fra...",1315.0,1074.0,1609.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2891,2019-01-01,3792,"Oracle Park, 24 Willie Mays Plaza, San Francis...",3760,"Powell BART Station, Market St and Powell St, ...",677.0,453.0,1012.0,NaN,NaN,...,572.0,426.0,769.0,NaN,NaN,NaN,730.0,467.0,1139.0,1
2642,2019-01-01,3396,"The Palace Of Fine Arts, 3601 Lyon St, San Fra...",3692,2nd Street and Stevenson Street (Montgomery BA...,1559.0,1233.0,1970.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1809.0,1551.0,2110.0,1
3005,2019-01-01,3396,"The Palace Of Fine Arts, 3601 Lyon St, San Fra...",3760,"Powell BART Station, Market St and Powell St, ...",1576.0,1279.0,1942.0,NaN,NaN,...,1545.0,1248.0,1912.0,NaN,NaN,NaN,1685.0,1427.0,1989.0,1
2531,2019-01-01,3692,2nd Street and Stevenson Street (Montgomery BA...,3792,"Oracle Park, 24 Willie Mays Plaza, San Francis...",517.0,274.0,973.0,NaN,NaN,...,396.0,248.0,630.0,311.0,184.0,525.0,571.0,297.0,1096.0,1
2672,2019-01-01,3394,"Fisherman's Wharf, San Francisco, CA",3692,2nd Street and Stevenson Street (Montgomery BA...,1125.0,756.0,1674.0,NaN,NaN,...,943.0,717.0,1240.0,761.0,631.0,918.0,1736.0,1011.0,2983.0,1
2078,2019-01-01,3394,"Fisherman's Wharf, San Francisco, CA",3603,"Embarcadero, San Francisco, CA",1124.0,670.0,1886.0,NaN,NaN,...,869.0,618.0,1221.0,643.0,508.0,814.0,1516.0,851.0,2699.0,1
1864,2019-01-01,3396,"The Palace Of Fine Arts, 3601 Lyon St, San Fra...",3603,"Embarcadero, San Francisco, CA",1450.0,1115.0,1886.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [31]:
barts_hotspots.describe()

,Origin Movement ID,Destination Movement ID,Daily Mean Travel Time (Seconds),Daily Range - Lower Bound Travel Time (Seconds),Daily Range - Upper Bound Travel Time (Seconds),AM Mean Travel Time (Seconds),AM Range - Lower Bound Travel Time (Seconds),AM Range - Upper Bound Travel Time (Seconds),PM Mean Travel Time (Seconds),PM Range - Lower Bound Travel Time (Seconds),...,Midday Mean Travel Time (Seconds),Midday Range - Lower Bound Travel Time (Seconds),Midday Range - Upper Bound Travel Time (Seconds),Evening Mean Travel Time (Seconds),Evening Range - Lower Bound Travel Time (Seconds),Evening Range - Upper Bound Travel Time (Seconds),Early Morning Mean Travel Time (Seconds),Early Morning Range - Lower Bound Travel Time (Seconds),Early Morning Range - Upper Bound Travel Time (Seconds),weekday
count,3258.000000,3258.000000,3200.000000,3200.000000,3200.000000,1792.000000,1792.000000,1792.000000,2330.000000,2330.000000,...,2531.000000,2531.000000,2531.000000,2232.000000,2232.000000,2232.000000,847.000000,847.000000,847.000000,3258.000000
mean,3606.166667,3606.166667,934.348125,698.232188,1263.199063,834.953125,641.654576,1097.446987,972.216738,739.224034,...,868.477282,668.650731,1141.673252,709.750896,541.439516,938.841398,493.866588,368.656434,672.506494,3.016575
std,160.628116,160.628116,334.460241,285.426149,432.028746,347.103206,288.793871,447.085859,363.293225,304.504823,...,325.855926,280.194978,435.570973,249.415298,216.879751,306.241767,178.493313,147.573987,255.790190,1.993319
min,3394.000000,3394.000000,320.000000,223.000000,443.000000,234.000000,172.000000,285.000000,266.000000,204.000000,...,291.000000,197.000000,381.000000,283.000000,160.000000,371.000000,226.000000,97.000000,289.000000,0.000000
25%,3396.000000,3396.000000,658.000000,469.750000,925.000000,585.000000,441.000000,764.750000,708.000000,513.250000,...,621.000000,454.000000,841.000000,500.000000,349.000000,704.000000,384.000000,269.000000,515.000000,1.000000
50%,3647.500000,3647.500000,878.500000,633.000000,1212.000000,766.000000,576.000000,1015.000000,925.500000,679.000000,...,798.000000,594.000000,1074.000000,671.000000,507.000000,900.500000,445.000000,330.000000,617.000000,3.000000
75%,3760.000000,3760.000000,1216.250000,962.000000,1547.000000,1038.250000,799.500000,1367.250000,1195.250000,932.000000,...,1068.000000,834.500000,1361.500000,876.250000,683.000000,1123.000000,562.000000,422.500000,774.500000,5.000000
max,3792.000000,3792.000000,2400.000000,1607.000000,5670.000000,3223.000000,2050.000000,5068.000000,3893.000000,2834.000000,...,3427.000000,2038.000000,6872.000000,1753.000000,1356.000000,2969.000000,1809.000000,1551.000000,2983.000000,6.000000


In [ ]:
with open():
    